In [1]:
import numpy as np
import re
import os
import h5py
from sklearn.preprocessing import LabelBinarizer
import keras
from keras import optimizers
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, add
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ZeroPadding2D
from keras import regularizers
from keras import backend as K

Using TensorFlow backend.


In [2]:
def load_dataset():
    train_dataset = h5py.File('dataset/hdf5_train_dataset.hdf5', 'r')
    train_set_x_orig = np.array(train_dataset['train_set_X'][:])
    train_set_y_orig = np.array(train_dataset['train_set_Y'][:])
    
    test_dataset = h5py.File('dataset/hdf5_test_dataset.hdf5', 'r')
    test_set_x_orig = np.array(test_dataset['test_set_X'][:])
    test_set_y_orig = np.array(test_dataset['test_set_Y'][:])
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig

In [3]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    X -- input data, of shape (input size, number of examples)(m, n_H, n_W, n_C)
    Y -- true 'label' vector (containing 0 if cat, 1 if non-cat) of shape (1, number of examples)(m,n_y)
    mini_batch_size -- size of minibatches, integer
    """
    m = X.shape()   #number of examples
    mini_batches = []
    
    #Step 1 -- Shuffle X and Y
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]
    
    #Step 2 -- partition of X and Y minus the end case
    num_complete_minibatches = math.floor(m/mini_batch_size)
    for i in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[i*mini_batch_size: i* mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[i*mini_batch_size: i*mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
        
    #handling the end case (last_mini_batch < mini_batch_size)
    if m%mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size:m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches*mini_batch_size:m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
        
    return mini_batches

In [4]:
def convert_to_one_hot(Y,C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [5]:
def forward_propagation_for_predict(X, parameters):
    
    """Implement Linear -> Relu -> Linear -> ReLu -> Softmax
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit 
    
    """
    #Retrieve the parameters from the dictionary 'parameters'
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.add(tf.matmul(W1,X), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(W2,A1), b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(W3,A2), b3)
    
    return Z3

In [ ]:
def predict(X, parameters):
    
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
 
    params = {"W1" : W1,
             "b1" : b1,
             "W2" : W2,
             "b2" : b2, 
             "W3" : W3,
              "b3" : b3
             }
    x = tf.placeholder("float", [12288,1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x:X})
    
    return prediction